Another pain point that is commonly brought up is the lack of "proper"
member-access specifiers.

While C++ has `public`, `private`, and `protected`, Python has underscore
prefixes. A single `_` for "please treat this as a private member" and two
`__` for "seriously, this is private!".
If you really want to keep people away, you can keep adding underscores.

How about we add real access specifiers and checks in Python?
This is going to be fun.

## Member Access Specifiers

Let's start with what we want to achieve.

In [ ]:
class Thing(CppClass):
    number: int

    def private_method():
        print("Private!")

    public()

    def Thing(number):
        this.number = number

    def public_method():
        print("Public!")
        print(this.number)
        this.private_method()

Once more, we're going to be building off the tools we've already used.

First, let's review type-creation again.

When Python defined a class, it starts by evaluating the code in the class
body. Once it is done, the defined members are passed into the type
constructor to create a new type.

While we're inside the function body, the code is executed like in any
other place.

Let's review the tools at our disposal:

1. Global state variables
2. Functions to change the global state
3. The ability to query the variables in the current scope
4. The ability to get scopes above us in the call stack
5. Function decorators
6. Context Managers

Let's start with the global state.
We're going to keep track of 2 things.
1. The current access specifier
2. The members defined with that specifier.

Whenever we call an access specifier, we look up the stack
and query the local variables. Then we store their names in one of 4 dictionaries:
1. Default access
2. Private Access
3. Protected Access
4. Public Access

With default existing to support both structs and classes.
When it is called again, we only add the _new_ names to the dicts.

In the type constructor, we collect all the names that were not assigned
yet, and set them to the current access specifier.

Then, for each function, we add an `__access__` attribute with it's matching
access specifier.

For members, we need to change the actual member class. It has to get the
access specifier when it is constructed and store it internally.

Now that we know which member has what access, we need to be able to check it.

The first step in checking is knowing which function called us and which
class it belongs to.

So you guessed it - another global stack! This time, a caller stack.
And yes, a caller context manager.
Whenever we call a method, we add the `(instance, method)` pair into the
stack. For free functions, we add `(None, function)`.

Lastly, we add an access check in `cpp_method`, and another access check
in the `CppMember` getter and setter.

In [ ]:
from enum import Enum, auto
import inspect


class Access(Enum):
    default = auto()
    private = auto()
    public = auto()
    protected = auto()


_access_info = {}
_current_access = Access.default


def set_access(access: Access):
    global _current_access
    for name in inspect.stack()[2].frame.f_locals:
        if name.startswith("__"):
            continue

        if name not in _access_info:
            _access_info[name] = _current_access

    _current_access = access


def public():
    set_access(Access.public)


def private():
    set_access(Access.private)


def protected():
    set_access(Access.protected)

In [ ]:
class CppMember:
    def __init__(self, access: Access):
        self.access = access
        ...

    ...


class CppClassMeta(type):
    ...

    @staticmethod
    def _create_members(dct):
        for name, type in dct.get("__annotations__", {}).items():
            member = CppMember(_access_info[name])
            dct[name] = member

    @staticmethod
    def _wrap_methods(dct):
        to_replace = {}

        for name, value in dct.items():
            if name.startswith("__"):
                continue

            if inspect.isroutine(value):
                value.__access__ = _access_info[name]
                to_replace[name] = cpp_method(value)

        dct.update(to_replace)

In [ ]:
_caller_stack = []


class CallerScope:
    def __init__(self, caller):
        _caller_stack.append(caller)

    def __enter__(self):
        pass

    def __exit__(self, exc_type, exc_val, exc_tb):
        _caller_stack.pop()


def get_caller():
    with suppress(IndexError):
        return _caller_stack[-1]
    return None


def cpp_method(f):
    ...
    with CallerScope(self):
        ...


def cpp_function(f):
    ...
    with CallerScope(f):
        ...

In [ ]:
def may_access(caller, target, required_access):
    if caller is None:
        # We're calling from pure-Python, so we allow all :P
        return True

    if required_access == access.Default:
        required_access = target.__default_access__

    if required_access == access.Public:
        return True

    if required_access == access.Protected:
        return isinstance(caller.instance, target.__class__)

    if required_access == access.Private:
        return type(caller) == type(target)

    # TODO: Add support for friend classes and free-functions!

    return False

def cpp_method(f):
    ...
    may_access(...)
    with CallerScope(self):
        ...


In [ ]:
class CppMember:
    def __get__(self, instance, owner=None):
        caller = get_caller()
        if not may_access(caller, instance, self.access):
            raise AccessError()
        ...

    def __set__(self, instance, value):
        caller = get_caller()
        if not may_access(caller, instance, self.access):
            raise AccessError()
        ...